In [7]:
import RandomForest as RF
import os
from datetime import datetime
import bz2
import _pickle as cPickle
import warnings
warnings.filterwarnings("ignore")

my_os = "mac" #win mac

separator = "\\"

if my_os == "mac":
    separator = "//"


today = datetime.now()
folderName = 'Experiments 3 '  + today.strftime("%d-%b-%Y %H_%M_%S")

if not os.path.exists(folderName):
    os.makedirs(folderName)

dataset = "kdd" # iris kdd  nsl_kdd  unsw_nb15 cic_ids_2017
problem = "AC" # AD AC
folder = "Experiments 2 subsets"

criterion_input = "entropy" #["gini", "entropy"]
ensemble_method = "SV"
method = "RF_S_DTs_A_All" #None RF_S_DTs_A_All RF_S_DTs_WA_All RF_S_DTs_A RF_S_DTs_WA

if dataset == "kdd" and problem == "AD" :
    # num_trees = 65
    criterion_input = 'entropy' #["gini", "entropy"]
    ensemble_method = "SV"

if dataset == "kdd" and problem == "AC" :
    # num_trees = 53
    criterion_input = 'entropy' #["gini", "entropy"]
    ensemble_method = "WV"

if dataset == "nsl_kdd" and problem == "AD" :
    # num_trees = 65
    criterion_input = 'gini' #["gini", "entropy"]
    ensemble_method = "SV"

if dataset == "nsl_kdd" and problem == "AC" :
    # num_trees = 31
    criterion_input = 'entropy' #["gini", "entropy"]
    ensemble_method = "SV"

if dataset == "unsw_nb15" and problem == "AD" :
    # num_trees = 93
    criterion_input = 'entropy' #["gini", "entropy"]
    ensemble_method = "WV"

if dataset == "unsw_nb15" and problem == "AC" :
    # num_trees = 61
    criterion_input = 'entropy' #["gini", "entropy"]
    ensemble_method = "SV"

if dataset == "cic_ids_2017" and problem == "AD" :
    criterion_input = 'entropy' #["gini", "entropy"]
    ensemble_method = "WV"
    # num_trees = 77

if dataset == "cic_ids_2017" and problem == "AC" :
    criterion_input = 'entropy' #["gini", "entropy"]
    ensemble_method = "WV"
    # num_trees = 95

no_runs = 5
start_from_run = 1

k = start_from_run

file_name_data_same = folder + separator +dataset+ separator + dataset +"_"+ problem + "_"
file_name_algorithms_same  = folder + separator +dataset+ separator + "Experiment_2_2_RF_"+dataset+"_"+problem+"_"+criterion_input+"_"+ensemble_method+"_"


wrong = 0
while k<=no_runs:
    try:
        file_name_data = file_name_data_same + str(k) + "_subsets_for_Experiment3.pbz2"
        f = bz2.BZ2File(file_name_data, "rb")
    except FileNotFoundError:
        wrong+=1
    k+=1

if wrong>0:
    print("Error: Make sure that you have the folder with results from Experiment 2 with the following path: 'Experiments 2 subsets//[dataset]//' and that it contains one file for each of 'k' runs with the following naming convention: '[dataset]_[AD/AC]_[k]_subsets_for_Experiment3.pbz2'")
else:
    k = start_from_run

    while k<=no_runs:

        file_name_data = file_name_data_same + str(k) + "_subsets_for_Experiment3.pbz2"
        f = bz2.BZ2File(file_name_data, "rb")


        dataTraining, targetTraining, subsetsTraining, dataValidation, targetValidation, subsetsValidation, dataTesting, targetTesting, subsetsTesting = cPickle.load(f)
        f.close()

        print("\n INFORMATION\n")
        print("Dataset: "+dataset)
        print("Number Training Cases: "+str(dataTraining.shape[0]))
        print("Number whole Testing Cases: "+str(dataTesting.shape[0])+"\n")


        file_name_algorithms = file_name_algorithms_same + str(k)+".pbz2"
        #f = open(file_name, "rb")
        f = bz2.BZ2File(file_name_algorithms, "rb")
        algorithms = cPickle.load(f)
        f.close()

        print("**********************************************")
        print()
        print("            EXPERIMENT 3")
        print()
        print("**********************************************")

        print("   RUN " + str(k))
        print("****************")
        print("   RF_S_DTs_A_All   ")
        print("****************")

        federatedLearnings = []
        for nT in range(1,len(algorithms)*algorithms[0].size+1):
            if dataset == "cic_ids_2017" and (nT % len(algorithms) != 0  or nT>500):
                continue
            numTreesPerRF = nT
            fl = RF.FederatedLearning()
            fl.mergeMethod1(algorithms, numTreesPerRF)
            fl.predict(dataTesting, targetTesting)
            print("numTrees: "+str(fl.size)+"     Accuracy: ",str(fl.statisticsRF.accuracy))
            if dataset == "cic_ids_2017":
                fl.trees = []
            federatedLearnings.append(fl)

        f = bz2.BZ2File(folderName+separator+"Experiment_3_RF_"+dataset+"_"+problem+"_"+criterion_input+"_"+ensemble_method+"_RF_S_DTs_A_All_"+str(k)+".pbz2", "wb")
        cPickle.dump((federatedLearnings),f)
        f.close()



        print("****************")
        print("   RF_S_DTs_WA_All")
        print("****************")

        federatedLearnings = []
        for nT in range(1,len(algorithms)*algorithms[0].size+1):
            if dataset == "cic_ids_2017" and (nT % len(algorithms) != 0  or nT>500):
                continue
            numTreesPerRF = nT
            fl = RF.FederatedLearning()
            fl.mergeMethod2(algorithms, numTreesPerRF)
            fl.predict(dataTesting, targetTesting)
            print("numTrees: "+str(fl.size)+"     Accuracy: ",str(fl.statisticsRF.accuracy))
            if dataset == "cic_ids_2017":
                fl.trees = []
            federatedLearnings.append(fl)

        f = bz2.BZ2File(folderName+separator+"Experiment_3_RF_"+dataset+"_"+problem+"_"+criterion_input+"_"+ensemble_method+"_RF_S_DTs_WA_All_"+str(k)+".pbz2", "wb")
        cPickle.dump((federatedLearnings),f)
        f.close()


        print("****************")
        print("   RF_S_DTs_A")
        print("****************")

        federatedLearnings = []
        for nT in range(1,algorithms[0].size+1):
            if dataset == "cic_ids_2017"  and nT*14>500:
                continue
            numTreesPerRF = nT
            fl = RF.FederatedLearning()
            fl.mergeMethod3(algorithms, numTreesPerRF)
            fl.predict(dataTesting, targetTesting)
            print("numTrees: "+str(fl.size)+"     Accuracy: ",str(fl.statisticsRF.accuracy))
            if dataset == "cic_ids_2017":
                fl.trees = []
            federatedLearnings.append(fl)


        f = bz2.BZ2File(folderName+separator+"Experiment_3_RF_"+dataset+"_"+problem+"_"+criterion_input+"_"+ensemble_method+"_RF_S_DTs_A_"+str(k)+".pbz2", "wb")
        cPickle.dump((federatedLearnings),f)
        f.close()

        print("****************")
        print("   RF_S_DTs_WA")
        print("****************")

        federatedLearnings = []
        for nT in range(1,algorithms[0].size+1):
            if dataset == "cic_ids_2017"  and nT*14>500:
                continue
            numTreesPerRF = nT
            fl = RF.FederatedLearning()
            fl.mergeMethod4(algorithms, numTreesPerRF)
            fl.predict(dataTesting, targetTesting)
            print("numTrees: "+str(fl.size)+"     Accuracy: ",str(fl.statisticsRF.accuracy))
            if dataset == "cic_ids_2017":
                fl.trees = []
            federatedLearnings.append(fl)

        f = bz2.BZ2File(folderName+separator+"Experiment_3_RF_"+dataset+"_"+problem+"_"+criterion_input+"_"+ensemble_method+"_RF_S_DTs_WA_"+str(k)+".pbz2", "wb")
        cPickle.dump((federatedLearnings),f)
        f.close()
        k+=1

Error: Make sure that you have the folder with results from Experiment 2 with the following path: 'Experiments 2 subsets//[dataset]//' and that it contains one file for each of 'k' runs with the following naming convention: '[dataset]_[AD/AC]_[k]_subsets_for_Experiment3.pbz2'
